In [1]:
import re
import codecs
import numpy as np
import struct
import hashlib
import os

from copy import copy
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def dict_of_words(): #opens files and makes dicts of words with posting lists
    
    files = os.listdir("./dataset/")
    urls = dict()
    
    all_texts_list = list()
    dicts = list()
    
    shift = 0
    vocabulary = 0
    
    for file in files:
        
        if file.find("lenta") < 0:
            continue
            
        with open("./dataset/" + file, 'rb') as f:
            file = f.read()
            
        f_text = file.decode('utf-8', errors='ignore')
        
        f_text = re.sub(r'[\x00\s]+', ' ', f_text)
        f_text = re.sub(r'[\x01\s]+', ' ', f_text)
        f_text = re.sub(r'[\x02\s]+', ' ', f_text)
        f_text = re.sub(r'[\x03\s]+', ' ', f_text)
        f_text = re.sub(r'[\x04\s]+', ' ', f_text)
        f_text = re.sub(r'[\x05\s]+', ' ', f_text)
        f_text = re.sub(r'[\x06\s]+', ' ', f_text)
        f_text = re.sub(r'[\x07\s]+', ' ', f_text)
        f_text = re.sub(r'[\x08\s]+', ' ', f_text)
        f_text = re.sub(r'[\x09\s]+', ' ', f_text)
        f_text = re.sub(r'[\x1a\s]+', ' ', f_text)
        f_text = re.sub(r'[\x13\s]+', ' ', f_text)
        
        f_text = f_text.lower()
        
        texts = re.findall(r"http[s]?://.*?http[s]?://", f_text)
        urls, texts_list = preprocess(texts, urls)
        all_texts_list += texts_list
        
        dicts.append((dict_maker(texts_list), shift))
        shift += len(texts_list)
        
        print("Urls number = {}".format(len(urls.keys())))
        
    return urls, dicts, all_texts_list

In [3]:
def preprocess(texts, urls = dict()): #returns url list and list of clear texts for one file
     
    text_list = list()
    cur_num = len(urls.keys())
    
    for i, text in enumerate(texts):
        
        text = text[:-7]
        url = re.findall(r"http[s]?:[^ ]*", text)
        text = text[len(url[0]):]
        
        urls[i + cur_num] = url
        text_list.append(text)
        
    return urls, text_list

In [4]:
def fibonacci(): #fibonacci generator
    a, b = 1, 2   
    while True:       
        yield a
        a, b = b, a + b

In [5]:
def add_new_bits(bit_seq, new_bits, length): #add new bits to bit sequence
    
    if bit_seq == 0:
        return new_bits
        
    bit_seq = bit_seq << 1
    bit_seq += 1
    
    bit_seq = bit_seq << length
    bit_seq += new_bits

    return bit_seq

In [6]:
def fib_encode(bit_seq, new_num): #encodes one number and concats it's bits seq to input bits sequence
    
    code = 0
    tmp_code = 1
    length = 1
    first_rank_flag =1
    
    while new_num != 0:
        gen = fibonacci()
        
        f = next(gen)
        prev = f
        
        while new_num - f > 0:
            
            length = length + 1 if first_rank_flag else length
            f, prev = next(gen), f
            tmp_code = tmp_code << 1
      
        if new_num - f != 0:
            
            tmp_code = tmp_code >> 1 
            new_num -= prev
            length -= 1
    
        else:
            new_num -= f
        
        first_rank_flag = 0
        code += tmp_code
        tmp_code = 1

    length = len(bin(code))-2
    return add_new_bits(bit_seq, code, length)

In [7]:
def fib_decode(bit_seq): #decodes sequence of bits, returns encoded list
    
    gen = fibonacci()
    lst = list()
    num = 0
    last = 0
    
    while bit_seq != 0:
        
        last, prev = bit_seq & 1, last
        bit_seq = bit_seq >> 1
        
        if last * prev == 1:
            
            lst.append(num)
            
            num = 0
            prev = 0
            last = 0
            gen = fibonacci()
            
            continue
            
        fib_val = next(gen)
        
        if last:
            num += fib_val

    lst.append(num)
    lst.reverse()
    return lst

In [8]:
def fib_encode_list(lst, b=0): #encode list of numbers

    for i in lst:
        b = fib_encode(b, i)
        
    return b

In [9]:
def inverse_index(X, words, idx): # function recieves count vectorizer's matrix, it's vocabulary and word index
                                  # returns hashed word, encoded posting list and word frequencies
    
    docs = np.where(X[:, idx] > 0)[0]

    vals = X[:, idx][np.where(X[:, idx] > 0)[0]]
    docs += 1 #as 0 can't be encoded

    docs_dif = np.zeros(docs.shape[0])
    docs_dif[0] = docs[0] 
    for i in range(1, docs.shape[0]):
        docs_dif[i] = docs[i] - docs[i-1]

    word = words[idx]

    hashed_word = hash(word)

    docs_dif_b = fib_encode_list(docs_dif)
    vals_b = fib_encode_list(vals)
    
    return hashed_word, docs_dif_b, vals_b

In [10]:
def docs_set(docs_dif): # restores posting list from list of documents differs
    
    docs_list = np.zeros(len(docs_dif))
    docs_list[0] = docs_dif[0]
    
    for i in range(1, len(docs_dif)):
        docs_list[i] = docs_list[i-1] + docs_dif[i]

    docs_list = docs_list.astype(int)
    
    return docs_list - 1

In [11]:
def convert(elem): # defines priority of element
    
    if elem == '&':
        return (2, '&')
    
    elif elem == '|':
        return (1, '|')
    
    elif elem == '(':
        return (-1, '(')
    
    elif elem == ')':
        return (-2, ')')
    
    else:
        hashed = hash(elem)
        return (0, hashed)

In [12]:
def poliz_maker(lst): # makes poliz from query list
    
    res = list()
    tmp_list = list()
    
    for elem in lst:
        
        pair = convert(elem)
        
        if pair[0] == 0:
            res.append(pair[1])
#             res.append(pair[1])
            
        elif pair[0] > 0:
            while (len(tmp_list) > 0) and (tmp_list[-1][0] >= pair[0]) and (tmp_list[-1][1] != '('):
                res.append(tmp_list.pop()[1])
            tmp_list.append(pair)
            
        elif pair[0] < 0:
            if pair[1] == '(':
                tmp_list.append(pair)
            else:
                while (len(tmp_list) > 0) and (tmp_list[-1][1] != '('):
                    res.append(tmp_list.pop()[1])
                tmp_list.pop()
    while (len(tmp_list) > 0):
        res.append(tmp_list.pop()[1])
        
    return res

In [13]:
def initialize_func(f):
    if f == '|':
        return lambda x, y: x | y
    if f == '&':
        return lambda x, y: x & y

In [14]:
def implement_poliz(lst): # processes poliz list, returns list of docs which responds to query
    i = 0
    while len(lst) != 1:
        i += 1

        if type(lst[i]) == type('|'):
            
            arg1 = lst.pop(i-2)
            i -= 1
            
            arg2 = lst.pop(i-1)
            i -= 1

            func = lst[i]
            
            if ((len(arg1[0]) > 0) and (arg1[0][0] == -1)) and ((len(arg2[0]) > 0) and (arg2[0][0] == -1)):
                
                docs = list()
                
            elif ((len(arg1[0]) > 0) and (arg1[0][0] == -1)):
                
                docs = arg2[0] if func == '|' else list()
                arg1 = (arg1[0], np.zeros(len(arg2[1])))
                
            elif ((len(arg2[0]) > 0) and (arg2[0][0] == -1)):
                
                docs = arg1[0] if func == '|' else list()
                arg2 = (arg2[0], np.zeros(len(arg1[1])))
                
            else:
                func = initialize_func(func)
                docs = list(func(set(arg1[0]), set(arg2[0])))

            words = np.zeros(len(docs))
            
            for j in range(len(docs)):
            
                words[j] = np.asarray(arg1[1])[np.where(arg1[0] == docs[j])].sum() + \
                           np.asarray(arg2[1])[np.where(arg2[0] == docs[j])].sum()

            lst[i] = (docs, words)
    
    return np.asarray(lst[0][0]), lst[0][1] 

In [15]:
def concat_bit_seqs(b1, b2): # concatination of two bits sequences
    
    b1 = b1 << 1
    b1 += 1
    
    b1 = b1 << len(bin(b2)) - 2
    b1 += b2
    
    return b1

In [16]:
def dict_maker(texts): # makes dict of texts list
    
    main_dict=dict()

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(texts)

    X = X.toarray()
    words = vectorizer.get_feature_names()

    for i in range(len(words)):
    
#         if i %10000 == 0:
#             print(i)

        res = inverse_index(X, words, i)
        if res[0] in main_dict.keys():
            
            docs = concat_bit_seqs(main_dict[res[0]][0], res[1])
            vals = concat_bit_seqs(main_dict[res[0]][1], res[2])
            main_dict[res[0]] = (docs, vals)
            
        else:
            main_dict[res[0]] = (res[1], res[2])
            
    return main_dict

In [17]:
def preprocess_query(string): # converts string representation of query to list
    
    string = re.sub(r'&', r' & ', string)
    string = re.sub(r'\|', r' | ', string)
    string = re.sub(r'\(', r' ( ', string)
    string = re.sub(r'\)', r' ) ', string)
    string = string.split()

    return string

In [18]:
def multy_dict_search(dicts, query, urls): # searches documents by query in file's dicts seperately
    
    expanded_query = preprocess_query(query)
    p_list = poliz_maker(expanded_query)
    
    res = list()
    words_num_array = np.zeros(0)
    docs_array = np.zeros(0)

    for d, shift in dicts:
        lst = copy(p_list)

        for i in range(len(lst)):
            
            if type(lst[i]) == type(0):
                
                if lst[i] in d.keys():
                    info = d[lst[i]]
                    lst[i] = (docs_set(fib_decode(info[0])), fib_decode(info[1]))
                
                else:
                    lst[i] = ([-1], 0)

        docs, words_num = implement_poliz(lst)
        docs += shift
#         print("docs, words = ", docs, words_num)
        docs_array = np.hstack((docs_array, docs))
        words_num_array = np.hstack((words_num_array, words_num))
#         print(docs_array, words_num_array)

    docs_array = docs_array[np.argsort(words_num_array)][::-1]
    words_num_array = np.sort(words_num_array)[::-1]

    for doc_id in docs_array:
        res.append((urls[doc_id], np.int(words_num_array[np.where(docs_array==doc_id)][0])))
           
    return res

In [63]:
class bulean_searcher: 
    
    def __init__(self):
        self.urls, self.dicts, self.texts = dict_of_words()
        
    def search(self, s=0, top=-1): # processes general workflow of searcher and prints results
        
        if type(s) == type(0):
            s = input()
        self.final = multy_dict_search(self.dicts, s, self.urls)
        
        if len(self.final) == 0:
            print("Ничего не найдено, пожалуйста переформулируйте ваш запрос.")
            return
        
        if top == -1:
            top = len(self.final)
            
        i = -1 
        
        for i, val in enumerate(self.final):
            
            url, words_num = val
            
            if i >= top:
                continue
            
            print(url[0], end='')
            print(' '*(55 - len(url[0])), end='')
            
            if ((words_num % 10 == 2) or (words_num % 10 == 3) or \
                (words_num % 10 == 4)) and (words_num not in range(11, 20)): # just printing right form of word
                
                print(words_num, "Совпадения")
                
            elif (words_num % 10 == 1):
                
                print(words_num, "Совпадение")
                
            else:
                
                print(words_num, "Совпадений")
        
        i += 1
        if ((i % 10 == 2) or (i % 10 == 3) or (i % 10 == 4)): # just printing right form of word

            print("Найдено {} страницы".format(i))

        elif (i % 10 == 1):

            print("Найденa {} страницa".format(i))

        else:

            print("Найдено {} страниц".format(i))        

In [64]:
bs = bulean_searcher()

Urls number = 655
Urls number = 1266
Urls number = 1860
Urls number = 2495
Urls number = 3119
Urls number = 3756
Urls number = 4361
Urls number = 5006


In [65]:
bs.search('политика & (путин | медведев)')

http://lenta.ru/articles/2008/12/04/putin/             49 Совпадений
http://lenta.ru/lib/14161216/full                      22 Совпадения
http://lenta.ru/conf/tsepliaeva                        10 Совпадений
http://lenta.ru/articles/2012/04/18/prokhorova/        8 Совпадений
http://lenta.ru/articles/2013/08/16/reaction/          5 Совпадений
http://lenta.ru/lib/14159417/full.htm                  5 Совпадений
http://lenta.ru/news/2005/09/14/kasianov2/             5 Совпадений
http://lenta.ru/lib/14160961/full                      4 Совпадения
http://lenta.ru/articles/2006/07/24/hloponin/          4 Совпадения
http://lenta.ru/conf/nzubarevich                       3 Совпадения
http://lenta.ru/articles/2012/04/30/perm/              3 Совпадения
http://lenta.ru/articles/2010/03/30/review/            3 Совпадения
http://lenta.ru/articles/2013/10/01/belarus/           2 Совпадения
http://bragin-sasha.livejournal.com/                   2 Совпадения
http://lenta.ru/articles/2015/08/28/rodina/  

In [66]:
bs.search('игра&карты | (теннис & кино)')

http://lenta.ru/articles/2014/11/07/kino7nov/          8 Совпадений
http://lenta.ru/articles/2009/12/16/nokia/             2 Совпадения
http://lenta.ru/articles/2011/03/09/zoich/             2 Совпадения
Найдено 3 страницы


In [67]:
bs.search('игра & приставка ')

http://lenta.ru/news/2012/12/03/ouya/                  2 Совпадения
Найденa 1 страницa


In [68]:
bs.search('(игра | приставка)& обезьяна')

Ничего не найдено, пожалуйста переформулируйте ваш запрос.


In [69]:
bs.search('игра | приставка | обезьяна', top=5)

http://lenta.ru/online/2010/12/08/zhispa               6 Совпадений
http://lenta.ru/articles/2009/06/09/infamous/          5 Совпадений
http://lenta.ru/news/2013/12/12/fable/                 5 Совпадений
http://lenta.ru/news/2011/01/17/cod/                   5 Совпадений
http://lenta.ru/news/2012/06/14/playdead/              4 Совпадения
Найдено 94 страницы


In [77]:
bs.search() #query from input

игра & футбол | нефть & газ
http://lenta.ru/articles/2008/12/04/putin/             13 Совпадений
http://lenta.ru/news/2008/08/05/gazprom/               12 Совпадений
http://lenta.ru/online/2010/12/08/zhispa               10 Совпадений
http://lenta.ru/news/2008/12/18/price/                 9 Совпадений
http://lenta.ru/articles/2015/03/06/gavrilov/          6 Совпадений
http://lenta.ru/news/2010/05/25/fall/                  5 Совпадений
http://lenta.ru/news/2010/06/01/first/                 4 Совпадения
http://lenta.ru/articles/2015/01/27/solar/             3 Совпадения
http://lenta.ru/news/2009/10/16/trade/                 3 Совпадения
http://lenta.ru/news/2006/03/16/marketreview/          2 Совпадения
http://lenta.ru/news/2008/01/28/cme/                   2 Совпадения
http://lenta.ru/articles/2011/03/09/zoich/             2 Совпадения
http://lenta.ru/news/2011/05/25/back/                  2 Совпадения
http://lenta.ru/news/2006/04/21/marketrewiew/          2 Совпадения
http://lenta.ru/n

In [72]:
bs.search('путин & трулвыоалов')

Ничего не найдено, пожалуйста переформулируйте ваш запрос.


In [76]:
bs.search('путин | трулвыоалов', top=3)

http://lenta.ru/articles/2008/12/04/putin/             48 Совпадений
http://lenta.ru/lib/14161216/full                      19 Совпадений
http://lenta.ru/articles/2013/11/21/pisateli/          14 Совпадений
Найдено 197 страниц
